<a href="https://colab.research.google.com/github/Ayush-mishra-0-0/NLP/blob/main/nlp_ass2_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch

In [4]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [6]:
squad

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5000
})

In [7]:
squad = squad.train_test_split(test_size=0.2)

In [8]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1000
    })
})

In [9]:
squad["train"][0]

{'id': '56cf38dcaab44d1400b88ea2',
 'title': 'New_York_City',
 'context': "The New York metropolitan area is home to a self-identifying gay and bisexual community estimated at 568,903 individuals, the largest in the United States and one of the world's largest. Same-sex marriages in New York were legalized on June 24, 2011 and were authorized to take place beginning 30 days thereafter.",
 'question': 'How many days after gay marriage was legalized were gay marriages allowed to take place?',
 'answers': {'text': ['30'], 'answer_start': [294]}}

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [12]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [14]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
training_args = TrainingArguments(
    output_dir="my_awesome_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-76d3147b044d>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,2.351514
2,2.700700,1.814079
3,2.700700,1.746326


TrainOutput(global_step=750, training_loss=2.253440633138021, metrics={'train_runtime': 579.3721, 'train_samples_per_second': 20.712, 'train_steps_per_second': 1.295, 'total_flos': 1175877900288000.0, 'train_loss': 2.253440633138021, 'epoch': 3.0})

In [16]:
trainer.save_model("my_awesome_qa_model")
tokenizer.save_pretrained("my_awesome_qa_model")

('my_awesome_qa_model/tokenizer_config.json',
 'my_awesome_qa_model/special_tokens_map.json',
 'my_awesome_qa_model/vocab.txt',
 'my_awesome_qa_model/added_tokens.json',
 'my_awesome_qa_model/tokenizer.json')

In [17]:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."

In [20]:
question = "where does shivam live"
context = "My name is shivam and i live in bhagalpur bihar.his cgpa is 9"

In the output of the `question-answerer` pipeline, `score` represents the confidence level of the model's answer, ranging between 0 and 1. A higher score indicates a greater confidence in the extracted answer being correct.


In [21]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="my_awesome_qa_model",device=0)
question_answerer(question=question, context=context)

{'score': 0.5040506720542908,
 'start': 32,
 'end': 47,
 'answer': 'bhagalpur bihar'}

### Visualizing Attention head using BertViz


In [23]:
!pip install bertviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 6.5 MB/s eta 0:00:00


In [24]:
from bertviz import head_view, model_view

def Visualize_attn_heads(cur_model, cur_tokenizer):
  # Define context and question
  context = "There are 45 iterations where one can succeed."
  question = "How many iterations can one succeed in?"

  # Tokenize question and context
  inputs = cur_tokenizer(question, context, return_tensors="pt")
  inputs = {k: v.to(DEVICE) for k, v in inputs.items()}  # Move inputs to device

  # Enable output of attention scores
  cur_model.config.output_attentions = True
  cur_model.eval()

  # Run the model to get outputs and attention scores
  with torch.no_grad():
      outputs = cur_model(**inputs)
      attentions = outputs.attentions  # Extract attention scores

  input_tokens = cur_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
  # layer_index = 1  # You can change this index to visualize different layers
  # if layer_index < len(attentions):
      # head_view(attentions, input_tokens, layer= layer_index )

  model_view(attentions, input_tokens)

Visualize_attn_heads(model, tokenizer)

DistilBertSdpaAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


<IPython.core.display.Javascript object>

### Pruning some heads

In [25]:
from transformers import AutoModelForQuestionAnswering
import torch

# Load your trained model
model = AutoModelForQuestionAnswering.from_pretrained("my_awesome_qa_model")

# Define the number of heads to prune from each layer
num_heads_to_prune = 2  # Adjust as needed

# Iterate through the layers and prune heads
for layer in model.distilbert.transformer.layer:
  if hasattr(layer, 'attention'):
    # Get the attention module
    attention_module = layer.attention
    # Get the number of heads
    num_heads = attention_module.n_heads
    # Prune the specified number of heads
    new_num_heads = num_heads - num_heads_to_prune
    attention_module.n_heads = new_num_heads
    # Adjust the weight matrices
    # Wrap the sliced tensors in nn.Parameter
    attention_module.q_lin.weight = torch.nn.Parameter(attention_module.q_lin.weight[:new_num_heads * attention_module.dim])
    attention_module.k_lin.weight = torch.nn.Parameter(attention_module.k_lin.weight[:new_num_heads * attention_module.dim])
    attention_module.v_lin.weight = torch.nn.Parameter(attention_module.v_lin.weight[:new_num_heads * attention_module.dim])
    attention_module.out_lin.weight = torch.nn.Parameter(attention_module.out_lin.weight[:attention_module.dim, :new_num_heads * attention_module.dim])

# Save the pruned model
model.save_pretrained("my_pruned_qa_model")

In [26]:
from transformers import pipeline, AutoTokenizer

def evaluate_pruned_model(pruned_model_path, question, context, tokenizer_path="my_awesome_qa_model"): # Add tokenizer_path
  """Evaluates a pruned question-answering model.

  Args:
    pruned_model_path: Path to the pruned model directory.
    question: The question to ask.
    context: The context containing the answer.
    tokenizer_path: Path to the directory containing the tokenizer. # Add this description

  Returns:
    The answer extracted by the model.
  """
  # Load the tokenizer from the original model path # Added this line
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

  question_answerer = pipeline("question-answering", model=pruned_model_path, tokenizer=tokenizer, device=0) # Pass the loaded tokenizer
  result = question_answerer(question=question, context=context)
  return result


# Example usage:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."
pruned_model_path = "my_pruned_qa_model"  # Replace with your actual path

result = evaluate_pruned_model(pruned_model_path, question, context)
result

{'score': 0.2095870077610016,
 'start': 58,
 'end': 95,
 'answer': '46 languages natural languages and 13'}

## Freezing Layers

In [27]:
# Load your trained model
model = AutoModelForQuestionAnswering.from_pretrained("my_awesome_qa_model")

# Freeze all layers except the last layer
for name, param in model.named_parameters():
    if 'layer.5' not in name:  # Freeze all layers except layer 5
        param.requires_grad = False

# Save the model with frozen layers
model.save_pretrained("my_frozen_qa_model")

def evaluate_frozen_model(frozen_model_path, question, context, tokenizer_path="my_awesome_qa_model"):
  """Evaluates a frozen question-answering model.

  Args:
    frozen_model_path: Path to the frozen model directory.
    question: The question to ask.
    context: The context containing the answer.
    tokenizer_path: Path to the directory containing the tokenizer.

  Returns:
    The answer extracted by the model.
  """
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
  question_answerer = pipeline("question-answering", model=frozen_model_path, tokenizer=tokenizer, device=0)
  result = question_answerer(question=question, context=context)
  return result


# Example usage:
question = "How many programming languages does BLOOM support?"
context = "BLOOM has 176 billion parameters and can generate text in 46 languages natural languages and 13 programming languages."
frozen_model_path = "my_frozen_qa_model"

result = evaluate_frozen_model(frozen_model_path, question, context)
result

{'score': 0.2095870077610016,
 'start': 58,
 'end': 95,
 'answer': '46 languages natural languages and 13'}

In [28]:
# Freeze different layers and evaluate performance
def freeze_layers(model, num_layers_to_freeze):
  """Freezes the bottom 'num_layers_to_freeze' layers of the model."""
  for name, param in model.named_parameters():
    if 'layer.' in name:
        # Extract layer number using regex to handle different naming patterns
        import re # Import regex library
        match = re.search(r'layer\.(\d+)', name)  # Search for "layer." followed by digits
        if match:
            layer_num = int(match.group(1))  # Get the captured digits as layer number
            if layer_num < num_layers_to_freeze:
                param.requires_grad = False

# Example: Freeze the bottom 2 layers
model = AutoModelForQuestionAnswering.from_pretrained("my_awesome_qa_model")
freeze_layers(model, 2)
model.save_pretrained("my_frozen_2_layers_qa_model")

# Evaluate the model with frozen layers
frozen_model_path = "my_frozen_2_layers_qa_model"
result = evaluate_frozen_model(frozen_model_path, question, context)
print(f"Results with 2 layers frozen: {result}")

# Example: Freeze the bottom 4 layers
model = AutoModelForQuestionAnswering.from_pretrained("my_awesome_qa_model")
freeze_layers(model, 4)
model.save_pretrained("my_frozen_4_layers_qa_model")

# Evaluate the model with frozen layers
frozen_model_path = "my_frozen_4_layers_qa_model"
result = evaluate_frozen_model(frozen_model_path, question, context)
print(f"Results with 4 layers frozen: {result}")


Results with 2 layers frozen: {'score': 0.2095870077610016, 'start': 58, 'end': 95, 'answer': '46 languages natural languages and 13'}
Results with 4 layers frozen: {'score': 0.2095870077610016, 'start': 58, 'end': 95, 'answer': '46 languages natural languages and 13'}


### Visualizing Attention head using BertViz


In [29]:
fre_tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [30]:
# Assuming you have a GPU available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def Visualize_attn_heads_frozen(cur_model, cur_tokenizer):
  # Define context and question
  context = "There are 45 iterations where one can succeed."
  question = "How many iterations can one succeed in?"

  # Tokenize question and context
  inputs = cur_tokenizer(question, context, return_tensors="pt")

  # Move both inputs and the model to the same device
  inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
  cur_model = cur_model.to(DEVICE)  # Move the model to the device

  # Enable output of attention scores
  cur_model.config.output_attentions = True
  cur_model.eval()

  # Run the model to get outputs and attention scores
  with torch.no_grad():
      outputs = cur_model(**inputs)
      attentions = outputs.attentions  # Extract attention scores

  input_tokens = cur_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
  # layer_index = 1  # You can change this index to visualize different layers
  # if layer_index < len(attentions):
      # head_view(attentions, input_tokens, layer= layer_index )

  model_view(attentions, input_tokens)


# Load your frozen model and tokenizer
frozen_model = AutoModelForQuestionAnswering.from_pretrained("my_frozen_qa_model")
frozen_tokenizer = AutoTokenizer.from_pretrained("my_awesome_qa_model")

# Visualize attention heads for the frozen model
Visualize_attn_heads_frozen(frozen_model, frozen_tokenizer)

<IPython.core.display.Javascript object>

## Using Adapter Modules in fine-tuning

In [31]:
adp_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
adp_model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

for param in adp_model.parameters():
    param.requires_grad = True

adp_model

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
     

In [ ]:
!pip install adapters

In [43]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from peft import AdaLoraModel, AdaLoraConfig
import torch

class QAInferencePipeline:
    def __init__(self, model_path):
        """
        Initialize the QA pipeline with a pre-trained model

        Args:
            model_path (str): Path to the saved model
        """
        try:
            # Load tokenizer and base model
            self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
            self.base_model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

            # Configure AdaLoRA
            adalora_config = AdaLoraConfig(
                peft_type="ADALORA",
                task_type="SEQ_2_SEQ_LM",
                init_r=12,
                target_r=8,
                beta1=0.85,
                beta2=0.85,
                tinit=200,
                tfinal=1000,
                deltaT=10,
                lora_alpha=32,
                target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],
                lora_dropout=0.1,
            )

            # Initialize AdaLoRA model
            self.model = AdaLoraModel(self.base_model, adalora_config, "default")

            # Load the trained weights
            self.model.load_state_dict(torch.load(f"{model_path}/pytorch_model.bin"))

            # Move model to GPU if available
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self.model = self.model.to(self.device)
            self.model.eval()

        except Exception as e:
            raise RuntimeError(f"Error initializing QA pipeline: {str(e)}")

    def get_answer(self, question, context):
        """
        Get answer for a question given the context

        Args:
            question (str): The question to answer
            context (str): The context containing the answer

        Returns:
            dict: Dictionary containing the answer and its position
        """
        try:
            # Tokenize input
            inputs = self.tokenizer(
                question,
                context,
                return_tensors="pt",
                max_length=384,
                truncation="only_second",
                padding=True
            )

            # Move inputs to the same device as model
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Get predictions
            with torch.no_grad():
                outputs = self.model(**inputs)

            # Get answer span
            answer_start = torch.argmax(outputs.start_logits)
            answer_end = torch.argmax(outputs.end_logits)

            # Decode answer
            answer_tokens = inputs['input_ids'][0][answer_start:answer_end+1]
            answer = self.tokenizer.decode(answer_tokens, skip_special_tokens=True)

            return {
                'answer': answer,
                'start': answer_start.item(),
                'end': answer_end.item()
            }

        except Exception as e:
            raise RuntimeError(f"Error during inference: {str(e)}")

# Example usage
if __name__ == "__main__":
    try:
        # Initialize pipeline
        qa_pipeline = QAInferencePipeline("/content/my_awesome_qa_model")

        # Example question and context
        question = "How many parameters does the model have?"
        context = "The model has 176 billion parameters and can generate text in 46 natural languages."

        # Get answer
        result = qa_pipeline.get_answer(question, context)
        print(f"Question: {question}")
        print(f"Answer: {result['answer']}")
        print(f"Answer span: {result['start']} to {result['end']}")

    except Exception as e:
        print(f"Error: {str(e)}")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Error: Error initializing QA pipeline: [Errno 2] No such file or directory: '/content/my_awesome_qa_model/pytorch_model.bin'


<ipython-input-43-e2bdb6297560>:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(f"{model_path}/pytorch_model.bin"))


In [37]:
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
    DataCollatorWithPadding
)
from datasets import Dataset
from torch.utils.data import DataLoader
from accelerate import Accelerator
from peft import AdaLoraConfig, AdaLoraModel
from transformers import AdamW, get_scheduler
import torch

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
base_model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

# Sample training data
train_data = [
    {
        "question": "How many parameters does the model have?",
        "context": "The model has 176 billion parameters and can generate text in 46 natural languages.",
        "answers": {"text": ["176 billion"], "answer_start": [13]},
    },
    # Add more examples here
]

# Preprocessing function
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find context boundaries
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Set positions
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Create datasets
train_dataset = Dataset.from_dict({
    "question": [example["question"] for example in train_data],
    "context": [example["context"] for example in train_data],
    "answers": [example["answers"] for example in train_data],
})

# Create validation dataset (you should add your own validation data)
eval_data = train_data[:1]  # Just using first example for demonstration
eval_dataset = Dataset.from_dict({
    "question": [example["question"] for example in eval_data],
    "context": [example["context"] for example in eval_data],
    "answers": [example["answers"] for example in eval_data],
})

# Preprocess datasets
tokenized_train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)
tokenized_eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=eval_dataset.column_names
)

# Create data loaders
data_collator = DataCollatorWithPadding(tokenizer)
train_dataloader = DataLoader(
    tokenized_train_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_eval_dataset,
    batch_size=8,
    collate_fn=data_collator
)

# Configure AdaLoRA - Updated target_modules for DistilBERT
adalora_config = AdaLoraConfig(
    peft_type="ADALORA",
    task_type="SEQ_2_SEQ_LM",
    init_r=12,
    target_r=8,
    beta1=0.85,
    beta2=0.85,
    tinit=200,
    tfinal=1000,
    deltaT=10,
    lora_alpha=32,
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin"],  # Updated for DistilBERT
    lora_dropout=0.1,
)

# Initialize AdaLoRA model
model = AdaLoraModel(base_model, adalora_config, "default")

# Initialize optimizer
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=3e-5)

# Initialize accelerator
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

# Training loop setup
num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Training loop
def train():
    model.train()
    for epoch in range(num_train_epochs):
        for batch in train_dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        # Evaluation
        model.eval()
        eval_loss = 0
        with torch.no_grad():
            for batch in eval_dataloader:
                outputs = model(**batch)
                eval_loss += outputs.loss.item()

        avg_eval_loss = eval_loss / len(eval_dataloader)
        print(f"Epoch {epoch+1}: Validation Loss = {avg_eval_loss:.4f}")

        model.train()

# Function for inference
def get_answer(question, context):
    inputs = tokenizer(
        question,
        context,
        return_tensors="pt",
        max_length=384,
        truncation="only_second",
        padding=True
    )

    with torch.no_grad():
        outputs = model(**inputs)

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits)

    answer = tokenizer.decode(inputs.input_ids[0][answer_start:answer_end+1])
    return answer

if __name__ == "__main__":
    train()

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Epoch 1: Validation Loss = 7.0642
Epoch 2: Validation Loss = 7.0635
Epoch 3: Validation Loss = 7.0631


In [40]:
import torch

def get_answer(question, context, model, tokenizer):
    # Move model to the same device as where it was trained
    device = next(model.parameters()).device

    # Create inputs using tokenizer
    inputs = tokenizer(
        question,
        context,
        return_tensors="pt",
        max_length=384,
        truncation="only_second",
        padding=True
    )

    # Move input tensors to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits)

    # Access input_ids directly from the tensor
    tokens = inputs['input_ids'][0][answer_start:answer_end+1].cpu()
    answer = tokenizer.decode(tokens, skip_special_tokens=True)
    return answer

# Example usage
def run_qa_example(model, tokenizer):
    question = "How many parameters does the model have?"
    context = "The model has 176 billion parameters and can generate text in 46 natural languages."

    try:
        answer = get_answer(question, context, model, tokenizer)
        print(f"Question: {question}")
        print(f"Answer: {answer}")
    except Exception as e:
        print(f"Error occurred: {str(e)}")

In [42]:
run_qa_example(model, tokenizer)



Question: How many parameters does the model have?
Answer: The model has 176 billion parameters and can generate text in 46 natural languages.

